## Requeriments

In [28]:
import pandas as pd
import numpy as np

## Carga

In [29]:
caminho_do_arquivo = '/content/drive/MyDrive/PI IV/PI/bases/SINAN/CHIKBR24.csv'
df = pd.read_csv(caminho_do_arquivo, sep=',', encoding='ISO-8859-1', low_memory=False)

## Entendimento dos dados

In [30]:
# Crição da variavel alvo
df['desfecho_grave'] = 0
df.loc[(df['HOSPITALIZ'] == 1) | (df['EVOLUCAO'] == 2), 'desfecho_grave'] = 1

# Definição das variaveis prediticas (features)
features_demograficas = [
    'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA'
]
features_clinicas = [
    'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'ARTRITE', 'ARTRALGIA'
]
features_comorbidades = [
    'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 'ACIDO_PEPT', 'AUTO_IMUNE'
]

colunas_selecionadas = features_demograficas + features_clinicas + features_comorbidades + ['desfecho_grave']

df_modelo = df[colunas_selecionadas].copy()

# Exploracao
print("\n" + "="*50)
print("VERIFICAÇÃO INICIAL DOS DADOS PARA O MODELO (ainda a)")
print("="*50)

print("\n[VERIFICAÇÃO 1: Balanceamento da Variável-Alvo 'desfecho_grave']")
print("0 = Não Grave (Curado, sem hospitalização)")
print("1 = Grave (Hospitalizado ou Óbito)")
print(df_modelo['desfecho_grave'].value_counts(dropna=False))
print("-"*50)

print("\n[VERIFICAÇÃO 2: Contagem de Dados Nulos (NaN) por Coluna]")
print(df_modelo.isnull().sum())
print("-"*50)

print("\n[VERIFICAÇÃO 3: Tipos de Dados das Colunas Selecionadas]")
df_modelo.info()
print("-"*50)

print("\n[VERIFICAÇÃO 4: Análise das Colunas Específicas de Dengue]")
colunas_dengue_suspeitas = ['GRAV_PULSO', 'GRAV_CONV', 'GRAV_HEMAT']
for coluna in colunas_dengue_suspeitas:
    if coluna in df.columns:
        contagem_nao_nulos = df[coluna].notna().sum()
        print(f"Coluna '{coluna}' tem {contagem_nao_nulos} valores preenchidos (não nulos).")
    else:
        print(f"Coluna '{coluna}' não encontrada no DataFrame.")
print("Se esses números forem muito baixos ou zero, confirma nossa hipótese de que não se aplicam à Chikungunya.")
print("="*50)


VERIFICAÇÃO INICIAL DOS DADOS PARA O MODELO (ainda a)

[VERIFICAÇÃO 1: Balanceamento da Variável-Alvo 'desfecho_grave']
0 = Não Grave (Curado, sem hospitalização)
1 = Grave (Hospitalizado ou Óbito)
desfecho_grave
0    390825
1     11983
Name: count, dtype: int64
--------------------------------------------------

[VERIFICAÇÃO 2: Contagem de Dados Nulos (NaN) por Coluna]
NU_IDADE_N           0
CS_SEXO              2
CS_GESTANT          43
CS_RACA              1
FEBRE             8339
MIALGIA           8339
CEFALEIA          8339
EXANTEMA          8339
ARTRITE           8339
ARTRALGIA         8339
DIABETES          8339
HEMATOLOG         8339
HEPATOPAT         8339
RENAL             8339
HIPERTENSA        8339
ACIDO_PEPT        8339
AUTO_IMUNE        8339
desfecho_grave       0
dtype: int64
--------------------------------------------------

[VERIFICAÇÃO 3: Tipos de Dados das Colunas Selecionadas]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402808 entries, 0 to 402807
Data columns

## Preparação dos Dados

In [31]:
df = pd.read_csv(caminho_do_arquivo, sep=',', encoding='ISO-8859-1', low_memory=False)

# Crição da variavel alvo
df['desfecho_grave'] = 0
df.loc[(df['HOSPITALIZ'] == 1) | (df['EVOLUCAO'] == 2), 'desfecho_grave'] = 1

# Definição das variaveis prediticas (features)
features_demograficas = [
    'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA'
]
features_clinicas = [
    'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'ARTRITE', 'ARTRALGIA'
]
features_comorbidades = [
    'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 'ACIDO_PEPT', 'AUTO_IMUNE'
]

colunas_selecionadas = features_demograficas + features_clinicas + features_comorbidades + ['desfecho_grave']

df_modelo = df[colunas_selecionadas].copy()

# === Limpeza e Pré-processamento ===

print(f"\nFormato original do df_modelo: {df_modelo.shape}")

# 1. Tratamento de Dados Faltantes: Remover as linhas com qualquer valor nulo
df_modelo.dropna(inplace=True)
print(f"Formato após remover linhas com dados nulos: {df_modelo.shape}")

# 2. Correção dos Tipos de Dados (Dtypes)
# Seleciona todas as colunas que são float e as converte para inteiro
colunas_float = df_modelo.select_dtypes(include='float64').columns
for col in colunas_float:
    df_modelo[col] = df_modelo[col].astype(np.int64)
print("\nTipos de dados corrigidos para inteiro.")

# 3. Preparação para o Machine Learning (One-Hot Encoding)
# Transforma colunas categóricas em colunas numéricas (0s e 1s)
colunas_para_encoder = ['CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'FEBRE', 'MIALGIA',
                        'CEFALEIA', 'EXANTEMA', 'ARTRITE', 'ARTRALGIA', 'DIABETES',
                        'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA',
                        'ACIDO_PEPT', 'AUTO_IMUNE']

df_final = pd.get_dummies(df_modelo, columns=colunas_para_encoder, drop_first=True)
print("Variáveis categóricas transformadas para o formato do modelo.")

# Verificacao
print("\n" + "="*50)
print("DATASET FINAL - PRONTO PARA A MODELAGEM")
print("="*50)

print("\nCinco primeiras linhas do DataFrame final:")
print(df_final.head())

print("\n\nInformações do DataFrame final:")
df_final.info()


Formato original do df_modelo: (402808, 18)
Formato após remover linhas com dados nulos: (394424, 18)

Tipos de dados corrigidos para inteiro.
Variáveis categóricas transformadas para o formato do modelo.

DATASET FINAL - PRONTO PARA A MODELAGEM

Cinco primeiras linhas do DataFrame final:
   NU_IDADE_N  desfecho_grave  CS_SEXO_I  CS_SEXO_M  CS_GESTANT_2  \
0        4037               1      False       True         False   
1        4037               0      False      False         False   
2        4029               0      False       True         False   
3        4056               0      False      False         False   
4        4058               0      False      False         False   

   CS_GESTANT_3  CS_GESTANT_4  CS_GESTANT_5  CS_GESTANT_6  CS_GESTANT_9  ...  \
0         False         False         False          True         False  ...   
1         False         False         False          True         False  ...   
2         False         False         False          T